# Restructuring Data in Pandas

This notebook is inspired from the textbook:

[Jake VanderPlas: Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/index.html)

In [5]:
import pandas as pd
import numpy as np
import chardet

## Agenda

- How to read data with different encodings
- How to find missing values in dataframes
- Dealing with missing values
- Combining dataframes
- Groupby method in Pandas
- Aggregation with groupby

## Reading Data - Encodings

[Download Data](https://www.kaggle.com/kwullum/fatal-police-shootings-in-the-us?select=PoliceKillingsUS.csv)

[Washington Post Repo](https://github.com/washingtonpost/data-police-shootings)

Package                            Version            
---------------------------------- -------------------
alabaster                          0.7.12             
anaconda-client                    1.7.2              
anaconda-navigator                 1.9.12             
anaconda-project                   0.8.3              
applaunchservices                  0.2.1              
appnope                            0.1.0              
appscript                          1.0.1              
argh                               0.26.2             
asn1crypto                         1.3.0              
astroid                            2.3.3              
astropy                            4.0                
atomicwrites                       1.3.0              
attrs                              19.3.0             
autopep8                           1.4.4              
Babel                              2.8.0              
backcall                           0.1.0              
backports.

Note: you may need to restart the kernel to use updated packages.


In [15]:
df = pd.read_csv('data/PoliceKillingsUS.csv', encoding = 'Windows-1252')
df.head(3)

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,02/01/15,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,02/01/15,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,03/01/15,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False


In [16]:
df.shape

(2535, 14)

### Encodings

In [17]:
before = "This is the euro symbol: €"

In [18]:
after = before.encode()

after

b'This is the euro symbol: \xe2\x82\xac'

### Solution to the encoding-decoding mess chardet library

[Chardet package](https://pypi.org/project/chardet/)

In [20]:
import sys
!{sys.executable} -m pip install chardet

[For more on installing packages from notebook](https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/)

In [22]:
import chardet
# look at the first ten thousand bytes to guess the character encoding
with open("data/PoliceKillingsUS.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(2000))

# check what the character encoding might be
print(result)

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


In [18]:
## try encoding = 'ascii'
## read the error carefully did it changed?

- [Kaggle - Dealing with Char Encodings](https://www.kaggle.com/rtatman/data-cleaning-challenge-character-encodings)

- [Python Standard Encodings](https://docs.python.org/3/library/codecs.html#standard-encodings)

In [23]:
df = pd.read_csv('data/PoliceKillingsUS.csv', encoding='Windows-1252')

In [24]:
print(df.shape)
df.head();

(2535, 14)


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2535 entries, 0 to 2534
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       2535 non-null   int64  
 1   name                     2535 non-null   object 
 2   date                     2535 non-null   object 
 3   manner_of_death          2535 non-null   object 
 4   armed                    2526 non-null   object 
 5   age                      2458 non-null   float64
 6   gender                   2535 non-null   object 
 7   race                     2340 non-null   object 
 8   city                     2535 non-null   object 
 9   state                    2535 non-null   object 
 10  signs_of_mental_illness  2535 non-null   bool   
 11  threat_level             2535 non-null   object 
 12  flee                     2470 non-null   object 
 13  body_camera              2535 non-null   bool   
dtypes: bool(2), float64(1), 

### Detecting Null Values

In [28]:
## find null values in each column
df.isna().sum().sum()

346

[Difference between `isna` and  `isnull`](https://datascience.stackexchange.com/questions/37878/difference-between-isna-and-isnull-in-pandas)

## Dealing with Missing Values

### Droping Rows with NaN values

In [27]:
## use of dropna with default
df.dropna()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,02/01/15,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,02/01/15,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,03/01/15,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,04/01/15,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,04/01/15,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2519,2805,Aries Clark,25/07/17,shot,undetermined,16.0,M,B,Marion,AR,False,undetermined,Not fleeing,False
2523,2808,Kesharn K. Burney,26/07/17,shot,vehicle,25.0,M,B,Dayton,OH,False,attack,Car,False
2525,2820,Deltra Henderson,27/07/17,shot,gun,39.0,M,B,Homer,LA,False,attack,Car,False
2533,2817,Isaiah Tucker,31/07/17,shot,vehicle,28.0,M,B,Oshkosh,WI,False,attack,Car,True


In [29]:
df.dropna(thresh = 10)

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,02/01/15,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,02/01/15,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,03/01/15,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,04/01/15,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,04/01/15,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2530,2822,Rodney E. Jacobs,28/07/17,shot,gun,31.0,M,NaN,Kansas City,MO,False,attack,Not fleeing,False
2531,2813,TK TK,28/07/17,shot,vehicle,NaN,M,NaN,Albuquerque,NM,False,attack,Car,False
2532,2818,Dennis W. Robinson,29/07/17,shot,gun,48.0,M,NaN,Melba,ID,False,attack,Car,False
2533,2817,Isaiah Tucker,31/07/17,shot,vehicle,28.0,M,B,Oshkosh,WI,False,attack,Car,True


In [32]:
df.dropna(subset = ['age','race'])

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,02/01/15,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,02/01/15,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,03/01/15,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,04/01/15,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,04/01/15,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2519,2805,Aries Clark,25/07/17,shot,undetermined,16.0,M,B,Marion,AR,False,undetermined,Not fleeing,False
2523,2808,Kesharn K. Burney,26/07/17,shot,vehicle,25.0,M,B,Dayton,OH,False,attack,Car,False
2525,2820,Deltra Henderson,27/07/17,shot,gun,39.0,M,B,Homer,LA,False,attack,Car,False
2533,2817,Isaiah Tucker,31/07/17,shot,vehicle,28.0,M,B,Oshkosh,WI,False,attack,Car,True


In [29]:
## how to drop only rows?

## how to drop columns?

## how about the other parameters: how, thresh, inplace, subset

### Filling Null Values

In [38]:
## droping might not be the best practice

#replace missing values with mean age value 

mean_age = int(df['age'].mean()) 
df['age'].fillna(mean_age, inplace = True)

In [42]:
df.race.value_counts()

W    1201
B     618
H     423
A      39
N      31
O      28
Name: race, dtype: int64

In [43]:
len(df)

2535

In [46]:
df.groupby(['city']).sum()

,id,age,signs_of_mental_illness,body_camera
city,,,,
Abbeville,2624,29.0,0.0,0.0
Abilene,2660,60.0,1.0,0.0
Abingdon,1254,67.0,0.0,0.0
Acworth,1158,50.0,1.0,0.0
Addison,1358,16.0,0.0,0.0
...,...,...,...,...
Yuba City,2521,43.0,0.0,0.0
Yulee,2623,51.0,0.0,0.0
Yuma,1695,24.0,0.0,0.0


In [44]:
#we ned to compare this to census data to see the likliness of different rae and police killings 

#hypothesis - police killings happenning more in east or west? 

## `Apply`, `Map` and `ApplyMap`

[Dataset](https://www.census.gov/data/tables/time-series/demo/popest/2010s-total-cities-and-towns.html)

In [31]:
## reading data from excel sheet

In [57]:
census = pd.read_excel('data/SUB-IP-EST2019-ANNRNK.xlsx', skiprows = 3, skipfooter = 5).drop('Unnamed: 0', axis =1)
census.tail()

,Unnamed: 1,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
783,"Lakewood city, Ohio",52131,52131,52026,51662,51400,51257,51088,50811,50528,50211,50012,49678
784,"Troy city, New York",50129,50162,50081,50020,49848,49921,49968,49776,49597,49433,49354,49154
785,"Saginaw city, Michigan",51508,51469,51363,51012,50728,50272,49798,49230,48902,48696,48310,48115
786,"Niagara Falls city, New York",50193,50031,49996,49760,49526,49328,49083,48769,48526,48249,48012,47720
787,"Charleston city, West Virginia",51400,51340,51274,50992,50858,50657,50133,49457,48789,48040,47172,46536


In [33]:
## use default? Is it ok?

## use skiprows. Are we done?

In [59]:
## dropping columns

## renaming columns

census.rename(columns = {'Unnamed: 1': 'place'})

,place,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,"New York city, New York",8175133,8175031,8190209,8272948,8346693,8396091,8433806,8463049,8469153,8437478,8390081,8336817
1,"Los Angeles city, California",3792621,3793139,3795512,3820876,3851202,3881622,3909901,3938568,3963226,3975788,3977596,3979576
2,"Chicago city, Illinois",2695598,2695652,2697477,2708114,2719141,2725731,2727066,2724344,2716723,2711069,2701423,2693976
3,"Houston city, Texas",2099451,2095517,2100280,2126032,2161593,2199391,2241826,2286908,2309544,2316750,2318573,2320268
4,"Phoenix city, Arizona",1445632,1446691,1449038,1469796,1499274,1526491,1555445,1583690,1612199,1633560,1654675,1680992
...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,"Lakewood city, Ohio",52131,52131,52026,51662,51400,51257,51088,50811,50528,50211,50012,49678
784,"Troy city, New York",50129,50162,50081,50020,49848,49921,49968,49776,49597,49433,49354,49154
785,"Saginaw city, Michigan",51508,51469,51363,51012,50728,50272,49798,49230,48902,48696,48310,48115
786,"Niagara Falls city, New York",50193,50031,49996,49760,49526,49328,49083,48769,48526,48249,48012,47720


### Apply Method
- works with dataframes and not series
- apply takes columns or rows

In [67]:
census[[2010,2011, 2012, 2013, 2014, 2015]].apply(lambda x: x.std(), axis=1)

0      103351.545722
1       54149.243158
2       11799.820090
3       70736.223456
4       51227.546397
           ...      
783       429.226747
784       111.958326
785       794.629914
786       448.081540
787       662.351996
Length: 788, dtype: float64

### Applymap Method
- treats every cell as input 

In [65]:
## try applymap as above
census[[2010,2011, 2012, 2013, 2014, 2015]].applymap(lambda x: x/1000)

,2010,2011,2012,2013,2014,2015
0,8190.209,8272.948,8346.693,8396.091,8433.806,8463.049
1,3795.512,3820.876,3851.202,3881.622,3909.901,3938.568
2,2697.477,2708.114,2719.141,2725.731,2727.066,2724.344
3,2100.280,2126.032,2161.593,2199.391,2241.826,2286.908
4,1449.038,1469.796,1499.274,1526.491,1555.445,1583.690
...,...,...,...,...,...,...
783,52.026,51.662,51.400,51.257,51.088,50.811
784,50.081,50.020,49.848,49.921,49.968,49.776
785,51.363,51.012,50.728,50.272,49.798,49.230
786,49.996,49.760,49.526,49.328,49.083,48.769


In [ ]:
## Splitting cities and states 

## Map
- works with series object

In [74]:
split_cities = df['Unnamed: 1'].map(lambda x: x.split(', ')[:-5])

KeyError: 'Unnamed: 1'

In [ ]:
## creating the abbrevated state names

In [68]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}


In [69]:
states = {values: key for key, values in states.items()}

In [70]:
census['abbr'] = census.state.map(lambda state: states[state])

AttributeError: 'DataFrame' object has no attribute 'state'

In [71]:
census.head(2)

,Unnamed: 1,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,"New York city, New York",8175133,8175031,8190209,8272948,8346693,8396091,8433806,8463049,8469153,8437478,8390081,8336817
1,"Los Angeles city, California",3792621,3793139,3795512,3820876,3851202,3881622,3909901,3938568,3963226,3975788,3977596,3979576


[More on the differences between `apply`,`map` and `applymap`](https://stackoverflow.com/questions/19798153/difference-between-map-applymap-and-apply-methods-in-pandas)

## Merging Datasets

This Part of the notebook is adopted from:

[PythonDataScienceHandbook - Merge -Join](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html)

In [75]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [76]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [77]:


df3 = pd.merge(df1, df2)
display('df1', 'df2','pd.merge(df1, df2)')



df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2)
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

In [78]:


df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')



df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

In [79]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


In [80]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

For more details please check this excellent notebook:

[Python Data Science Handbook - Merge-Join](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html)

And the `pandas` documentation

[Pandas - Merging](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)

In [81]:
df_merged = pd.merge(census[['city', 'state', 2016]], df, on = 'city', how = 'inner')

KeyError: "['city', 'state'] not in index"

In [ ]:
df_merged.head()

## Groupby

In [ ]:
grouped = df_merged.groupby(by = 'city')
grouped.get_group('New York').race.value_counts()

In [ ]:
grouped[2016].mean()

In [ ]:
grouped.aggregate(['min', np.median, max])['age']

In [ ]:
my_func = lambda x: x.max() - x.min()

In [ ]:
grouped.aggregate({'date': 'max', 'age': 'min', 2016: my_func})

In [ ]:
grouped = df_merged.groupby(by = ['city', 'state_x'])


In [ ]:
grouped.get_group(('New York', 'New York')).race.value_counts()

In [ ]:
grouped[2016].mean()

In [ ]:
(grouped['city'].count()/grouped[2016].mean()).sort_values(ascending =False)

In [ ]:
grouped.get_group('Toledo')

## Resources

[More on encodings](https://realpython.com/python-encodings-guide/)